In [ ]:
using Pipe

In [ ]:
normInf(vec::Tuple{Int64,Int64}) = map(abs, vec) |> Base.splat(max)

normalizeInf(vec::Tuple{Int64,Int64}) = map(x -> sign(x) * min(abs(x), 1), vec)

displayKnots(knots::Vector{Tuple{Int64,Int64}}) =
  begin
    nRows = 28
    nCols = 28
    board = fill('_', (nRows, nCols))

    for i in length(knots):-1:1
      (x, y) = knots[i]
      board[div(nRows, 2)-y, div(nCols, 2)+x+1] = '0' + i - 1
    end

    stringFormVec = Vector{String}()
    for i in 1:nRows
      strToCreate = ""
      for j in 1:nCols
        strToCreate *= board[i, j]
      end
      push!(stringFormVec, strToCreate)
    end

    stringForm = map(string, board)
    display(stringFormVec)
  end

In [ ]:
offsetFromDirection = Dict('U' => (0, 1), 'D' => (0, -1), 'L' => (-1, 0), 'R' => (1, 0))

parseInputLine(line::String) =
  begin
    splitted = split(line)
    direction = splitted[1] |> first
    (dx, dy) = offsetFromDirection[direction]
    count = parse(Int64, splitted[2])
    return (count * dx, count * dy)
  end

In [ ]:
moves = Vector{Tuple{Int64,Int64}}()
for move in eachline("input.txt")
  @pipe move |> parseInputLine |> push!(moves, _)
end

In [ ]:
# task both
task2 = true
reachedPositions = Set{Tuple{Int64,Int64}}([(0, 0)])
knotsCount = task2 ? 10 : 2
knotPositions = Vector{Tuple{Int64,Int64}}()
newKnotPositions = Vector{Tuple{Int64,Int64}}()
for _ in 1:knotsCount
  push!(knotPositions, (0, 0))
  push!(newKnotPositions, (0, 0))
end

for move in moves
  (dx, dy) = normalizeInf(move)
  for _ in 1:normInf(move)
    newKnotPositions[1] = knotPositions[1] .+ (dx, dy)

    for i in 1:knotsCount-1
      if normInf(knotPositions[i+1] .- newKnotPositions[i]) > 1
        newKnotPositions[i+1] = knotPositions[i+1] .+ normalizeInf(newKnotPositions[i] .- knotPositions[i+1])
      else
        newKnotPositions[i+1] = knotPositions[i+1]
      end
    end

    knotPositions = copy(newKnotPositions)
    push!(reachedPositions, knotPositions[knotsCount])
  end
end

reachedPositions |> length